In [1]:
import os
os.chdir("../")
os.getcwd()

'/home/izam/coding/Text-Summarizer'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    """
    A class to manage configuration settings.
    """
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH) -> None:
        """
        Initializes the ConfigurationManager object.

        Args:
            config_filepath (str, optional): File path for the main configuration file.
            params_filepath (str, optional): File path for the parameters file.
        """

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """
        Retrieves the data ingestion configuration.

        Returns:
            DataIngestionConfig: Object containing data ingestion configuration settings.
        """
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )

        return data_ingestion_config

In [5]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [7]:
class DataIngestion:
    """
    A class for handling data ingestion tasks.

    Attributes:
        config (DataIngestionConfig): Configuration object for data ingestion.

    Methods:
        __init__: Initializes the DataIngestion object.
        download_file: Downloads the data file from the specified source URL.
        extract_zip_file: Extracts the contents of a zip file into the specified directory.

    Usage:
        data_ingestion = DataIngestion(config)
        data_ingestion.download_file()
        data_ingestion.extract_zip_file()

    """
    def __init__(
            self,
            config: DataIngestionConfig
            ) -> None:
        """
        Initializes the DataIngestion object.

        Args:
            config (DataIngestionConfig): Configuration object for data ingestion.
        """
        self.config = config

    def download_file(self):
        """
        Downloads the data file from the specified source URL.

        Returns:
            None
        """
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename=self.config.local_data_file,
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    def extract_zip_file(self):
        """
        Extracts the contents of a zip file into the specified directory.

        Returns:
            None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-02 00:54:39,603: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-02 00:54:39,604: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-02 00:54:39,604: INFO: common: created directory at: artifacts]
[2024-02-02 00:54:39,605: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-02 00:54:42,442: INFO: 2114299802: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3298:4862:8187F2:9D3E88:65BBAF40
Accept-Ranges: bytes
Date: Thu, 01 Feb 2024 19:24:41 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10250-MAA
X-Cache: HIT